<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/GPT2_QA_Finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import transformers
print(transformers.__version__)

4.47.1


Huggingface trainer and generate

In [3]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset
import torch

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Tie lm_head weights if missing
if model.lm_head.weight.shape[0] != model.transformer.wte.weight.shape[0]:
    model.tie_weights()

# Add padding token to tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Example QA dataset
qa_data = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
]

# Preprocess dataset
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer:"
    target_text = example["answer"]
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=50)
    targets = tokenizer(target_text, truncation=True, padding="max_length", max_length=50)

    inputs["labels"] = targets["input_ids"]
    inputs["attention_mask"] = inputs["attention_mask"]
    return inputs

# Convert dataset to Huggingface Dataset object
dataset = Dataset.from_list(qa_data)
tokenized_dataset = dataset.map(preprocess_data, remove_columns=["question", "answer"])

# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2_qa_finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to=[],  # Disable W&B or any reporting
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()

# Save the model
model.save_pretrained("./gpt2_qa_finetuned")
tokenizer.save_pretrained("./gpt2_qa_finetuned")

# Test the model
def generate_answer(question, model, tokenizer, device="cuda"):
    model.to(device)  # Move the model to the specified device
    input_text = f"Question: {question}\nAnswer:"
    inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)  # Move inputs to the same device
    outputs = model.generate(inputs, max_length=100, num_beams=5, early_stopping=True, pad_token_id=tokenizer.eos_token_id)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Determine device
device = "cuda" if torch.cuda.is_available() else "cpu"

question = "What is the capital of France?"
answer = generate_answer(question, model, tokenizer)
print(answer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-3-5238537375dd>:55: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,5.542056
2,No log,3.124552
3,No log,2.559626


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Question: What is the capital of France?
Answer: The capital of France is the capital of France.


Generate from Scratch

In [4]:
def generate_answer(question, model, tokenizer, max_length=50, device="cuda"):
    """
    Generate an answer using greedy search.
    Args:
        question (str): The input question.
        model (GPT2LMHeadModel): The GPT-2 model.
        tokenizer (GPT2Tokenizer): The tokenizer.
        max_length (int): Maximum length of the generated text.
        device (str): Device to run the model on ('cuda' or 'cpu').

    Returns:
        str: The generated answer.
    """
    # Move the model to the correct device
    model.to(device)
    model.eval()  # Set model to evaluation mode

    # Prepare input
    input_text = f"Question: {question}\nAnswer:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Start generating
    generated_ids = input_ids
    for _ in range(max_length):
        # Get logits for the next token
        outputs = model(input_ids=generated_ids)
        logits = outputs.logits

        # Select the token with the highest probability (greedy search)
        next_token_id = torch.argmax(logits[:, -1, :], dim=-1).unsqueeze(0)

        # Append the predicted token to the sequence
        generated_ids = torch.cat([generated_ids, next_token_id], dim=-1)

        # Stop if the model predicts the end-of-sequence token
        if next_token_id.item() == tokenizer.eos_token_id:
            break

    # Decode the generated tokens to text
    answer = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return answer


# Determine device
device = "cuda" if torch.cuda.is_available() else "cpu"

question = "What is the capital of France?"
answer = generate_answer(question, model, tokenizer)
print(answer)



Question: What is the capital of France?
Answer: The capital of France is the capital of France.


Trainer from Scratch using Loss inside Model

In [15]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import Dataset
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm import tqdm
import random
import numpy as np

# Seed setting function
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Set the seed for reproducibility
seed = 42
set_seed(seed)

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Tie lm_head weights if missing
if model.lm_head.weight.shape[0] != model.transformer.wte.weight.shape[0]:
    model.tie_weights()

# Add padding token to tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Example QA dataset
qa_data = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
]

# Preprocess dataset
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer:"
    target_text = example["answer"]
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=50)
    targets = tokenizer(target_text, truncation=True, padding="max_length", max_length=50)

    inputs["labels"] = targets["input_ids"]
    return inputs

# Convert dataset to Huggingface Dataset object
dataset = Dataset.from_list(qa_data)
tokenized_dataset = dataset.map(preprocess_data, remove_columns=["question", "answer"])

# Define data loaders
batch_size = 2
train_loader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=True)

# Define optimizer and device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
def train_model(model, train_loader, optimizer, device, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        epoch_loss = 0
        for batch in tqdm(train_loader):
            input_ids = torch.stack(batch["input_ids"]).to(device)
            attention_mask = torch.stack(batch["attention_mask"]).to(device)
            labels = torch.stack(batch["labels"]).to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch {epoch + 1} Loss: {epoch_loss / len(train_loader)}")

# Validation loop (optional)
def validate_model(model, val_loader, device):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader):
            input_ids = torch.stack(batch["input_ids"]).to(device)
            attention_mask = torch.stack(batch["attention_mask"]).to(device)
            labels = torch.stack(batch["labels"]).to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    print(f"Validation Loss: {val_loss / len(val_loader)}")

# Train the model
train_model(model, train_loader, optimizer, device)

# Save the model
model.save_pretrained("./gpt2_qa_finetuned")
tokenizer.save_pretrained("./gpt2_qa_finetuned")

def generate_answer(question, model, tokenizer, max_length=50, device="cuda"):
    """
    Generate an answer using greedy search.
    Args:
        question (str): The input question.
        model (GPT2LMHeadModel): The GPT-2 model.
        tokenizer (GPT2Tokenizer): The tokenizer.
        max_length (int): Maximum length of the generated text.
        device (str): Device to run the model on ('cuda' or 'cpu').

    Returns:
        str: The generated answer.
    """
    # Move the model to the correct device
    model.to(device)
    model.eval()  # Set model to evaluation mode

    # Prepare input
    input_text = f"Question: {question}\nAnswer:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Start generating
    generated_ids = input_ids
    for _ in range(max_length):
        # Get logits for the next token
        outputs = model(input_ids=generated_ids)
        logits = outputs.logits

        # Select the token with the highest probability (greedy search)
        next_token_id = torch.argmax(logits[:, -1, :], dim=-1).unsqueeze(0)

        # Append the predicted token to the sequence
        generated_ids = torch.cat([generated_ids, next_token_id], dim=-1)

        # Stop if the model predicts the end-of-sequence token
        if next_token_id.item() == tokenizer.eos_token_id:
            break

    # Decode the generated tokens to text
    answer = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return answer


# Determine device
device = "cuda" if torch.cuda.is_available() else "cpu"

question = "What is the capital of France?"
answer = generate_answer(question, model, tokenizer)
print(answer)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Epoch 1/10


100%|██████████| 1/1 [00:00<00:00,  8.14it/s]


Epoch 1 Loss: 7.131683826446533
Epoch 2/10


100%|██████████| 1/1 [00:00<00:00,  9.42it/s]


...

Epoch 8 Loss: 4.1919264793396
Epoch 9/10


100%|██████████| 1/1 [00:00<00:00, 12.37it/s]


Epoch 9 Loss: 3.343966484069824
Epoch 10/10


100%|██████████| 1/1 [00:00<00:00, 12.23it/s]


Epoch 10 Loss: 2.8702731132507324
Question: What is the capital of France?
Answer: The capital of France is the capital of France


Trainer and Loss from Scratch

In [54]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import Dataset
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
import os
import random
import numpy as np

# Seed setting function
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Set the seed for reproducibility
seed = 42
set_seed(seed)

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Tie lm_head weights if missing
if model.lm_head.weight.shape[0] != model.transformer.wte.weight.shape[0]:
    model.tie_weights()

# Add padding token to tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Example QA dataset
qa_data = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
]

# Preprocess dataset
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer:"
    target_text = example["answer"]
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=50)
    targets = tokenizer(target_text, truncation=True, padding="max_length", max_length=50)

    inputs["labels"] = targets["input_ids"]
    return inputs

# Convert dataset to Huggingface Dataset object
dataset = Dataset.from_list(qa_data)
tokenized_dataset = dataset.map(preprocess_data, remove_columns=["question", "answer"])

# Define data loaders
batch_size = 2
train_loader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=True)

# Define optimizer, criterion, and device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# Validation loop
def validate_model(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            input_ids = torch.stack(batch["input_ids"]).to(device)
            attention_mask = torch.stack(batch["attention_mask"]).to(device)
            labels = torch.stack(batch["labels"]).to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Reshape logits and labels for CrossEntropyLoss
            shift_logits = logits[..., :-1, :].contiguous()  # Shift logits
            shift_labels = labels[..., 1:].contiguous()  # Shift labels
            loss = criterion(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            val_loss += loss.item()

    return val_loss / len(val_loader)

# Training loop
def train_model(model, train_loader, optimizer, criterion, device, num_epochs=10, save_dir="./best_model"):
    best_val_loss = float("inf")
    os.makedirs(save_dir, exist_ok=True)

    model.train()
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        epoch_loss = 0
        for batch in tqdm(train_loader):
            input_ids = torch.stack(batch["input_ids"]).to(device)
            attention_mask = torch.stack(batch["attention_mask"]).to(device)
            print('batch["labels"]:', batch["labels"][0].shape)
            labels = torch.stack(batch["labels"], dim=0).to(device)
            print('labels:', labels.shape)
            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Reshape logits and labels for CrossEntropyLoss
            shift_logits = logits[..., :-1, :].contiguous()  # Shift logits
            shift_labels = labels[..., 1:].contiguous()  # Shift labels
            loss = criterion(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        # print(f"Epoch {epoch + 1} Loss: {epoch_loss / len(train_loader)}")
        avg_train_loss = epoch_loss / len(train_loader)
        print(f"Epoch {epoch + 1} Training Loss: {avg_train_loss}")

        # Validate after each epoch
        val_loss = validate_model(model, train_loader, criterion, device)
        print(f"Epoch {epoch + 1} Validation Loss: {val_loss}")

        # Save the best model based on validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            model.save_pretrained(save_dir)
            tokenizer.save_pretrained(save_dir)
            print(f"Saved best model with Validation Loss: {val_loss}")

# Train the model
train_model(model, train_loader, optimizer, criterion, device)

# Save the model
model.save_pretrained("./gpt2_qa_finetuned")
tokenizer.save_pretrained("./gpt2_qa_finetuned")

def generate_answer(question, model, tokenizer, max_length=50, device="cuda"):
    """
    Generate an answer using greedy search.
    Args:
        question (str): The input question.
        model (GPT2LMHeadModel): The GPT-2 model.
        tokenizer (GPT2Tokenizer): The tokenizer.
        max_length (int): Maximum length of the generated text.
        device (str): Device to run the model on ('cuda' or 'cpu').

    Returns:
        str: The generated answer.
    """
    # Move the model to the correct device
    model.to(device)
    model.eval()  # Set model to evaluation mode

    # Prepare input
    input_text = f"Question: {question}\nAnswer:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Start generating
    generated_ids = input_ids
    for _ in range(max_length):
        # Get logits for the next token
        outputs = model(input_ids=generated_ids)
        logits = outputs.logits

        # Select the token with the highest probability (greedy search)
        next_token_id = torch.argmax(logits[:, -1, :], dim=-1).unsqueeze(0)

        # Append the predicted token to the sequence
        generated_ids = torch.cat([generated_ids, next_token_id], dim=-1)

        # Stop if the model predicts the end-of-sequence token
        if next_token_id.item() == tokenizer.eos_token_id:
            break

    # Decode the generated tokens to text
    answer = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return answer


# Determine device
device = "cuda" if torch.cuda.is_available() else "cpu"

question = "What is the capital of France?"

best_model_dir = './best_model'
model = GPT2LMHeadModel.from_pretrained(best_model_dir)
tokenizer = GPT2Tokenizer.from_pretrained(best_model_dir)

answer = generate_answer(question, model, tokenizer)
print(answer)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Epoch 1/10


100%|██████████| 1/1 [00:00<00:00,  5.91it/s]


batch["labels"]: torch.Size([2])
labels: torch.Size([50, 2])
Epoch 1 Training Loss: 11.76418399810791


Validating: 100%|██████████| 1/1 [00:00<00:00, 22.84it/s]


...

100%|██████████| 1/1 [00:00<00:00,  8.73it/s]


batch["labels"]: torch.Size([2])
labels: torch.Size([50, 2])
Epoch 10 Training Loss: 3.63531494140625


Validating: 100%|██████████| 1/1 [00:00<00:00, 38.58it/s]

Epoch 10 Validation Loss: 2.935006856918335


Saved best model with Validation Loss: 2.935006856918335
Question: What is the capital of France?
Answer: The capital of France is Paris.
The capital of France is Paris.
The capital of France is Paris.
The capital of France is Paris.
The capital of France is Paris.
The capital of France is Paris.
The capital


In [15]:
def generate_answer(question, model, tokenizer, max_length=50, device="cuda"):
    """
    Generate an answer using greedy search.
    Args:
        question (str): The input question.
        model (GPT2LMHeadModel): The GPT-2 model.
        tokenizer (GPT2Tokenizer): The tokenizer.
        max_length (int): Maximum length of the generated text.
        device (str): Device to run the model on ('cuda' or 'cpu').

    Returns:
        str: The generated answer.
    """
    # Move the model to the correct device
    model.to(device)
    model.eval()  # Set model to evaluation mode

    # Prepare input
    input_text = f"Question: {question}\nAnswer:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
    print('input_text:', input_text)
    print('input_ids:', input_ids)
    print('input_ids length:', input_ids.shape)
    print('input last token decode:', tokenizer.decode(input_ids[0][-1], skip_special_tokens=True))
    # Start generating
    generated_ids = input_ids
    for _ in range(max_length):
        # Get logits for the next token
        outputs = model(input_ids=generated_ids)
        logits = outputs.logits
        print('logits:', logits.shape)

        # Select the token with the highest probability (greedy search)
        next_token_id = torch.argmax(logits[:, -1, :], dim=-1).unsqueeze(0)
        print('next_token_id:', next_token_id)
        print('next_token_id decoder:', tokenizer.decode(next_token_id[0], skip_special_tokens=True))

        prev_id = torch.argmax(logits[:, -2, :], dim=-1).unsqueeze(0)
        print('prev_id decoder:', tokenizer.decode(prev_id[0], skip_special_tokens=True))

        # Append the predicted token to the sequence
        generated_ids = torch.cat([generated_ids, next_token_id], dim=-1)

        # Stop if the model predicts the end-of-sequence token
        if next_token_id.item() == tokenizer.eos_token_id:
            break

    # Decode the generated tokens to text
    answer = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return answer


# Determine device
device = "cuda" if torch.cuda.is_available() else "cpu"

question = "What is the capital of France?"

best_model_dir = './best_model'
model = GPT2LMHeadModel.from_pretrained(best_model_dir)
tokenizer = GPT2Tokenizer.from_pretrained(best_model_dir)

answer = generate_answer(question, model, tokenizer)
print(answer)

input_text: Question: What is the capital of France?
Answer:
input_ids: tensor([[24361,    25,  1867,   318,   262,  3139,   286,  4881,    30,   198,
         33706,    25]], device='cuda:0')
input_ids length: torch.Size([1, 12])
input last token decode: :
logits: torch.Size([1, 12, 50257])
next_token_id: tensor([[383]], device='cuda:0')
next_token_id decoder:  The
prev_id decoder: :
logits: torch.Size([1, 13, 50257])
next_token_id: tensor([[3139]], device='cuda:0')
next_token_id decoder:  capital
prev_id decoder:  The
logits: torch.Size([1, 14, 50257])
next_token_id: tensor([[286]], device='cuda:0')
next_token_id decoder:  of
prev_id decoder:  capital
logits: torch.Size([1, 15, 50257])
next_token_id: tensor([[4881]], device='cuda:0')
next_token_id decoder:  France
prev_id decoder:  of
logits: torch.Size([1, 16, 50257])
next_token_id: tensor([[318]], device='cuda:0')
next_token_id decoder:  is
prev_id decoder:  France
logits: torch.Size([1, 17, 50257])
next_token_id: tensor([[262]], d

In [50]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import Dataset
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
import os
import random
import numpy as np

# Seed setting function
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Set the seed for reproducibility
seed = 42
set_seed(seed)

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Tie lm_head weights if missing
if model.lm_head.weight.shape[0] != model.transformer.wte.weight.shape[0]:
    model.tie_weights()

# Add padding token to tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Example QA dataset
qa_data = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
]

# Preprocess dataset
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer:"
    target_text = example["answer"]
    # inputs = tokenizer(input_text, truncation=True )
    # targets = tokenizer(target_text, truncation=True)
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=20)
    targets = tokenizer(target_text, truncation=True, padding="max_length", max_length=20)

    inputs["labels"] = targets["input_ids"]
    return inputs

# Convert dataset to Huggingface Dataset object
dataset = Dataset.from_list(qa_data)
tokenized_dataset = dataset.map(preprocess_data, remove_columns=["question", "answer"])

# Define data loaders
batch_size = 2
train_loader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=True)

# Define optimizer, criterion, and device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)


# Training loop
def train_model(model, train_loader, optimizer, criterion, device, num_epochs=1, save_dir="./best_model"):
    best_val_loss = float("inf")
    os.makedirs(save_dir, exist_ok=True)

    model.train()
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        epoch_loss = 0
        for batch in tqdm(train_loader):
            input_ids = torch.stack(batch["input_ids"]).to(device)
            attention_mask = torch.stack(batch["attention_mask"]).to(device)
            labels = torch.stack(batch["labels"]).to(device)
            print('input_ids:', input_ids.shape) #[20, 2]
            # print('input_ids decoder:', tokenizer.decode(input_ids.squeeze(), skip_special_tokens=True))
            print('labels:', labels.shape) #[20, 2]
            # print('labels decoder:', tokenizer.decode(labels.squeeze(), skip_special_tokens=True))
            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits #[20, 2, 50257]
            print('logits:', logits.shape)

            # Reshape logits and labels for CrossEntropyLoss
            shift_logits = logits[..., :-1, :].contiguous()  # Shift logits [20, 1, 50257]
            shift_labels = labels[..., 1:].contiguous()  # Shift labels [20, 1]
            print('shift_logits:',shift_logits.shape)
            print('shift_labels:',shift_labels.shape)

            # print('shift_labels:', labels[..., 1:].shape, labels[..., 1:])
            # print('shift_labels decoder:', tokenizer.decode(labels[..., 1:].squeeze(), skip_special_tokens=True))

            loss = criterion(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            break

# Train the model
train_model(model, train_loader, optimizer, criterion, device)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Epoch 1/1


  0%|          | 0/1 [00:00<?, ?it/s]

input_ids: torch.Size([20, 2])
labels: torch.Size([20, 2])
logits: torch.Size([20, 2, 50257])
shift_logits: torch.Size([20, 1, 50257])
shift_labels: torch.Size([20, 1])


In [46]:
for batch in train_loader:
    # print(batch["labels"])
    print(torch.stack(batch["labels"]).shape)
    # print(torch.stack(batch["labels"]))
    # print(torch.stack(batch["labels"])[..., 1:])
    print(torch.stack(batch["labels"])[..., 1:].shape)
    break

torch.Size([50, 2])
torch.Size([50, 1])


In [14]:
arr = np.ones((3,1))
arr[0,0] = 1
arr[1,0] = 2
arr[2,0] = 3
arr, arr.shape, arr[-2,0]

(array([[1.],
        [2.],
        [3.]]),
 (3, 1),
 2.0)